In [ ]:
using Revise

using DataFrames

using GCTGMT
using Plot

using FeatureSetEnrichment

In [ ]:
use_style!()

;

## make_benchmark

In [ ]:
println(make_benchmark("card A2K"))

println(make_benchmark("random 100 10"))

# println(make_benchmark("myc"))

## sum_h_absolute_n_m

In [ ]:
sum_h_absolute_n_m(Float64.([-2, -1, 0, 1, 2]), Float64.([1, 1, 0, 0, 1]))

## score_set

In [ ]:
element_, score_, set_element_ = make_benchmark("card A2")

;

In [ ]:
score_set(element_, score_, set_element_)

## score_set_new

In [ ]:
score_set_new(element_, score_, set_element_)

## score_sets

In [ ]:
score_sets(
    "data/element_x_sample.tsv",
    ["./data/h.all.v7.1.symbols.gmt", "./data/c2.all.v7.1.symbols.gmt"],
    ".",
)